# Effectuer un classement des établissements selon leur niveau de bruit dans le référentiel Structure

objectif : produire un tableau avec les colonnes suivantes : nom de portail, nb structure, % de bruit

- obtenir un tableau de tous les portails
- réduire aux portails institutionnels
- trouver les structId des portails identifiés
    - explorer l'API AureHal Structure pour avoir l'id des établissements à partir d'une requête par nom
    - créer une fonction qui donne à partir du nom de l'établissement leur id
    - ajouter au tableau des établissements une colonne avec leur id
- pour chaque portail identifié extraire nb structure valide, non valide, % bruit 
    - créer une fonction qui donne : nb structure valide et non valide à partir d'un structId
    - ajouter les champs nb_struct_valid et nb_struct_no_valid au tableau des établissements
- calculer le pourcetange de bruit et l'intégrer au tableau
- retirer les etab où nous n'avons pas de structure valide, trier par noise en décroissant

## Obtenir la liste des portails HAL

In [24]:
import pandas as pd, json, requests

In [25]:
# la doc du referentiel des portails : https://api.archives-ouvertes.fr/docs/ref/resource/instance

In [26]:
raw = requests.get("https://api.archives-ouvertes.fr/ref/instance").json()
raw

{'response': {'numFound': 181,
  'docs': [{'id': '9',
    'code': 'tel',
    'name': 'TEL - Thèses en ligne',
    'url': 'https://tel.archives-ouvertes.fr'},
   {'id': '1',
    'code': 'hal',
    'name': 'Archive ouverte HAL',
    'url': 'https://hal.archives-ouvertes.fr'},
   {'id': '13',
    'code': 'archivesic',
    'name': '@rchiveSIC',
    'url': 'https://archivesic.ccsd.cnrs.fr'},
   {'id': '10',
    'code': 'jeannicod',
    'name': 'Institut | Nicod',
    'url': 'https://jeannicod.ccsd.cnrs.fr'},
   {'id': '5',
    'code': 'tematice',
    'name': "archive-EduTice - Education et technologies de l'information et de la communication",
    'url': 'https://edutice.archives-ouvertes.fr'},
   {'id': '44',
    'code': 'memsic',
    'name': "memSIC -  mémoires de master en Sciences de l'information et de la communication",
    'url': 'https://memsic.ccsd.cnrs.fr'},
   {'id': '6',
    'code': 'democrite',
    'name': 'IN2P3 - Institut national de physique nucléaire et de physique des part

In [27]:
# récupérer la liste de portails et la transformer en df
portails = pd.DataFrame(raw['response']['docs'])
portails

,id,code,name,url
0,9,tel,TEL - Thèses en ligne,https://tel.archives-ouvertes.fr
1,1,hal,Archive ouverte HAL,https://hal.archives-ouvertes.fr
2,13,archivesic,@rchiveSIC,https://archivesic.ccsd.cnrs.fr
3,10,jeannicod,Institut | Nicod,https://jeannicod.ccsd.cnrs.fr
4,5,tematice,archive-EduTice - Education et technologies de...,https://edutice.archives-ouvertes.fr
5,44,memsic,memSIC - mémoires de master en Sciences de l'...,https://memsic.ccsd.cnrs.fr
6,6,democrite,IN2P3 - Institut national de physique nucléair...,http://hal.in2p3.fr
7,2,inria,Inria,https://hal.inria.fr
8,7,halshs,HAL-SHS - Sciences de l'Homme et de la Société,https://halshs.archives-ouvertes.fr
9,8,artxiker,Artxiker,https://artxiker.ccsd.cnrs.fr


In [28]:
print(f"nb de portails {len(portails)}")

nb de portails 181


## Réduire aux protails institutionnels

In [29]:
pd.set_option('display.max_rows', None) #pour afficher toutes les rows de la df

In [30]:
filtre_name = portails["name"].str.contains(
    "univ|ecole|école|inst|inria|inserm|labo|cea|supelec|agro|cnam|enssib|ENS|CIrad|Observatoire|INSA|MNHN|inshea|lyon",
case = False, regex=True)
filtre_url = portails["url"].str.contains("univ|cnrs|sciencespo", case = False, regex=True)
#portails[filtre_name].sort_values(by = "name")
portail_etab = portails[filtre_name].sort_values(by = "name")
portail_etab

,id,code,name,url
89,92,afrique,Agence universitaire de la Francophonie,https://hal-auf.archives-ouvertes.fr
61,67,agroparistech,AgroParisTech,https://hal-agroparistech.archives-ouvertes.fr
140,5852,agrosup-dijon,AgroSup Dijon,https://hal-agrosup-dijon.archives-ouvertes.fr
83,90,agrocampus-ouest,Agrocampus Ouest,https://hal-agrocampus-ouest.archives-ouvertes.fr
101,110,amu,Aix-Marseille Université,https://hal-amu.archives-ouvertes.fr
23,26,cea,CEA - Commissariat à l’énergie atomique,https://hal-cea.archives-ouvertes.fr
147,6477,u-paris-seine,CY Cergy Paris Université,https://hal-cyu.archives-ouvertes.fr/
17,20,cirad,Cirad,http://hal.cirad.fr
152,6911,cnam,Cnam - Conservatoire national des arts et métiers,https://hal-cnam.archives-ouvertes.fr
36,39,dumas,DUMAS - Dépôt Universitaire de Mémoires Après ...,https://dumas.ccsd.cnrs.fr


## Trouver les structId des portails identifiés

In [31]:
# documentation référentiel structure https://api.archives-ouvertes.fr/docs/ref/?resource=structure&schema=fields#fields

In [115]:
def get_structId_from_name(name) : 
    """extrait le structID du nom de structure spécifié en entrée """
    # recherche dans acronyme et name
    # utiliser name_t car insensible à la casse et accents
    # memo : https://api.archives-ouvertes.fr/ref/structure/?fq=type_s:institution&q=(name_s:cirad OR acronym_s:Cirad)
    
    url = "https://api.archives-ouvertes.fr/ref/structure/?fq=type_s:institution&fq=valid_s:VALID"
    req = requests.get(url + f"&q=(name_t:\"{name}\" OR acronym_t:\"{name}\" )")
    r = req.json()
    try : 
        return r["response"]["docs"][0]["docid"]
    except : 
        print("pb ", name)

In [116]:
get_structId_from_name("Université Clermont Auvergne")

pb  Université Clermont Auvergne


In [117]:
portail_etab["structId"] = portail_etab.apply(lambda row : get_structId_from_name(row["name"]), axis = 1)

pb  CEA - Commissariat à l’énergie atomique
pb  Cnam - Conservatoire national des arts et métiers
pb  DUMAS - Dépôt Universitaire de Mémoires Après Soutenance
pb  EM-LYON Business School
pb  ENAC - Ecole Nationale de l'Aviation Civile
pb  ENS de Lyon
pb  EPHE - École pratique des hautes études
pb  Ecole Centrale Paris
pb  Groupe des Écoles Nationales d'Économie et Statistiques
pb  Hal Lumière Lyon2
pb  Haut Conseil de l’évaluation de la recherche et de l’enseignement supérieur
pb  ICP - Institut Catholique de Paris
pb  IN2P3 - Institut national de physique nucléaire et de physique des particules
pb  INERIS - Institut National de l’EnviRonnement Industriel et des RisqueS
pb  INSU - Institut national des sciences de l'Univers
pb  IRD - Institut de recherche pour le développement
pb  IRSN - Institut de Radioprotection et de Sûreté Nucléaire
pb  Inalco - Institut National des Langues et Civilisations Orientales
pb  Inrap - Institut national de recherches archéologiques préventives
pb  Inse

In [118]:
portail_etab.sample(5)

,id,code,name,url,structId,nb_struct_valid,nb_struct_no_valid,noise
17,20,cirad,Cirad,http://hal.cirad.fr,11574.0,60.0,1.0,2.0
3,10,jeannicod,Institut | Nicod,https://jeannicod.ccsd.cnrs.fr,NaN,NaN,NaN,NaN
161,7463,u-bordeaux-montaigne,Université Bordeaux Montaigne,https://hal-u-bordeaux-montaigne.archives-ouve...,412629.0,29.0,4.0,14.0
93,100,saga,Université Grenoble Alpes,https://hal.univ-grenoble-alpes.fr,NaN,19.0,86.0,453.0
121,4638,insep,"Insep - Institut national du sport, de l'exper...",https://hal-insep.archives-ouvertes.fr,NaN,NaN,NaN,NaN


In [119]:
# calculer taux de détection de structId de porail

In [120]:
print( f"Des structId ont été trouvés pour { len( portail_etab[ portail_etab['structId'].notna()] ) / len(portail_etab) } des étab.") 

Des structId ont été trouvés pour 0.5615384615384615 des étab.


## Créer une fonction qui donne : nb structure valide et non valide à partir d'un structId

In [121]:
def get_struct_nb(structId, valid):
    # si l'établissement n'a pas de structId alors on ne le traite pas
    if not pd.isna(structId) :  
        url = "https://api.archives-ouvertes.fr/ref/structure/?"
        req = requests.get(url+ f"&q=parentDocid_i:{int(structId)}&fq=valid_s:{valid}")
        req = req.json()
        return req["response"]["numFound"]    

In [122]:
# test de la foncion
get_struct_nb(172265,"INCOMING")

28

## Ajouter les champs "nb structure valide" et "nb structure non valide" au tableau des établissements

In [123]:
portail_etab["nb_struct_valid"] = portail_etab.apply(lambda row : get_struct_nb(row['structId'], "VALID"), axis = 1)

In [124]:
portail_etab.sample(5)

,id,code,name,url,structId,nb_struct_valid,nb_struct_no_valid,noise
168,7808,inrae,"Institut National de Recherche en Agriculture,...",https://hal.inrae.fr,NaN,NaN,NaN,NaN
100,107,ema,École nationale supérieure des mines d'Albi-Ca...,https://hal-mines-albi.archives-ouvertes.fr,469216.0,3.0,0.0,0.0
7,2,inria,Inria,https://hal.inria.fr,300009.0,59.0,0.0,0.0
177,8821,u-picardie,Université de Picardie Jules Verne,https://hal-u-picardie.archives-ouvertes.fr,300258.0,52.0,20.0,38.0
134,5512,univ-paris-lumieres,Université Paris Lumières,https://hal-univ-paris-lumieres.archives-ouver...,NaN,NaN,NaN,NaN


In [125]:
portail_etab["nb_struct_no_valid"] = portail_etab.apply(lambda row : get_struct_nb(row['structId'], "INCOMING"), axis = 1)

In [126]:
portail_etab.sample(3)

,id,code,name,url,structId,nb_struct_valid,nb_struct_no_valid,noise
152,6911,cnam,Cnam - Conservatoire national des arts et métiers,https://hal-cnam.archives-ouvertes.fr,NaN,NaN,NaN,NaN
94,101,univ-tln,Université de Toulon,https://hal-univ-tln.archives-ouvertes.fr,303091.0,28.0,14.0,50.0
144,6183,univ-pau-pays-adour,Université de Pau et des Pays de l'Adour,https://hal-univ-pau.archives-ouvertes.fr,301085.0,35.0,20.0,57.0


## calculer le pourcentage de bruit et l'intégrer au tableau

In [127]:
portail_etab["noise"] = round(portail_etab["nb_struct_no_valid"] / portail_etab["nb_struct_valid"] * 100, 0)

In [128]:
len(portail_etab)

130

In [129]:
## netoyer le tableay (nb struct = 0) et afficher le classement

In [130]:
portail_noise = portail_etab[ 
    (portail_etab["nb_struct_valid"].notna()) & 
    (portail_etab["noise"].notna()) 
]
len(portail_noise)

72

In [131]:
# conserver que les etab où structId > 0
portail_noise = portail_noise[ portail_noise["nb_struct_valid"] >0 ]
portail_noise

,id,code,name,url,structId,nb_struct_valid,nb_struct_no_valid,noise
61,67,agroparistech,AgroParisTech,https://hal-agroparistech.archives-ouvertes.fr,148117.0,27.0,2.0,7.0
140,5852,agrosup-dijon,AgroSup Dijon,https://hal-agrosup-dijon.archives-ouvertes.fr,415469.0,20.0,3.0,15.0
83,90,agrocampus-ouest,Agrocampus Ouest,https://hal-agrocampus-ouest.archives-ouvertes.fr,108028.0,24.0,50.0,208.0
101,110,amu,Aix-Marseille Université,https://hal-amu.archives-ouvertes.fr,198056.0,182.0,16.0,9.0
147,6477,u-paris-seine,CY Cergy Paris Université,https://hal-cyu.archives-ouvertes.fr/,1003413.0,26.0,0.0,0.0
17,20,cirad,Cirad,http://hal.cirad.fr,11574.0,60.0,1.0,2.0
157,7267,enssib,ENSSIB,https://hal-enssib.archives-ouvertes.fr,43938.0,2.0,2.0,100.0
57,64,ensieta,ENSTA Bretagne,https://hal-ensta-bretagne.archives-ouvertes.fr,301846.0,6.0,0.0,0.0
70,77,ensta,ENSTA Paris,https://hal-ensta-paris.archives-ouvertes.fr/,300065.0,6.0,1.0,17.0
160,7464,enc,Ecole Nationale des Chartes,https://hal-enc.archives-ouvertes.fr,307249.0,3.0,1.0,33.0


In [132]:
# trier par noise décroissant
portail_noise.sort_values(by = "noise", ascending = False)

,id,code,name,url,structId,nb_struct_valid,nb_struct_no_valid,noise
50,55,univ-paris13,Université Paris 13,https://hal-univ-paris13.archives-ouvertes.fr,15786.0,1.0,73.0,7300.0
162,7468,enva,École Nationale vétérinaire d'Alfort,https://hal-enva.archives-ouvertes.fr,34256.0,12.0,70.0,583.0
156,7264,univ-angers,Université d'Angers,https://hal.univ-angers.fr,74911.0,56.0,131.0,234.0
43,49,univ-nantes,Université de Nantes,http://hal.univ-nantes.fr,93263.0,72.0,150.0,208.0
83,90,agrocampus-ouest,Agrocampus Ouest,https://hal-agrocampus-ouest.archives-ouvertes.fr,108028.0,24.0,50.0,208.0
64,71,hcl,Hospices Civils de Lyon,https://hal-hcl.archives-ouvertes.fr,547850.0,1.0,2.0,200.0
88,91,univ-rennes1,Université de Rennes 1,https://hal-univ-rennes1.archives-ouvertes.fr,105160.0,106.0,189.0,178.0
163,7580,u-pec,Université Paris-Est Créteil Val-de-Marne,https://hal.u-pec.fr/,420786.0,95.0,146.0,154.0
103,109,insa-rennes,INSA Rennes,https://hal-insa-rennes.archives-ouvertes.fr,117606.0,7.0,10.0,143.0
10,11,inserm,Inserm,https://www.hal.inserm.fr,303623.0,602.0,802.0,133.0


---
Allez plus loin : 
    
* améliorer le matching nom de portail <-> structId
* ajouter des iterations : recherches les "structures rouges" non plus au niveau de la structure parente, mais dans toute les structures filles.